In [ ]:
#@markdown ## 1. Clone repo and install requirements
!git clone https://github.com/hzwer/arXiv2020-RIFE
!mkdir /content/arXiv2020-RIFE/train_log
%cd /content/arXiv2020-RIFE/train_log
!gdown --id 1APIzVeI-4ZZCEuIRE1m6WYfSCaOsi_7_
!7z e RIFE_trained_model_v3.6.zip
%cd /content/arXiv2020-RIFE/
!gdown --id 1i3xlKb7ax7Y70khcTcuePi6E7crO_dFc
!pip3 install -r requirements.txt

In [ ]:
#@markdown ## 2. Prepare Folder
import os.path
from os import path
import shutil

%cd /content/arXiv2020-RIFE

print("Current Directory: " + str(os.popen("pwd").read()))
if path.exists("./input_video"):
  shutil.rmtree('./input_video')
  os.system("mkdir input_video")
else:
  os.system("mkdir input_video")

if path.exists("./output"):
  shutil.rmtree('./output')
  os.system("mkdir output")
else:
  os.system("mkdir output")

if path.exists("./input_video/chunks"):
  shutil.rmtree('./input_video/chunks')
  os.system("mkdir input_video/chunks")
else:
  os.system("mkdir input_video/chunks")

if path.exists("./input_video/output_chunks"):
  shutil.rmtree("./input_video/output_chunks")
  os.system("mkdir input_video/output_chunks")
else:
  os.system("mkdir input_video/output_chunks")

In [ ]:
#@markdown ## 3. Split video into chunks (duration in second)
import os
fileName = "input.mp4" #@param {type: "string"}
durationPerChunk = 1 #@param {type: "number"}
%cd /content/arXiv2020-RIFE/input_video
os.popen(f"ffmpeg -i {fileName} -acodec copy -f segment -segment_time {durationPerChunk} -vcodec copy -reset_timestamps 1 -map 0 ./chunks/chunk%d.mp4").read()

In [ ]:
#@markdown ## 4. Start Processing
!nvidia-smi
import os, os.path

exp = 2 #@param {type: "number"}
fps = 60 #@param {type: "number"}
UHD = True #@param {type: "boolean"}

%cd /content/arXiv2020-RIFE

chunks = len([c for c in os.listdir('./input_video/chunks') if os.path.isfile(os.path.join("./input_video/chunks", c))])

for i in range(chunks):
  print(f"Processing {i}/{chunks}")
  if UHD:
    os.system(f"python3 inference_video.py --exp={str(exp)} --UHD --fps=str(fps) --output=./input_video/output_chunks/chunk{i}.mp4 --video=./input_video/chunks/chunk{i}.mp4 --skip")
  else:
    os.system(f"python3 inference_video.py --exp={str(exp)} --fps=str(fps) --output=./input_video/output_chunks/chunk{i}.mp4 --video=./input_video/chunks/chunk{i}.mp4 --skip")
print("Done!")

In [ ]:
#@markdown ## Merge output chunks into 1 file (Optional)
import os, os.path
%cd /content/arXiv2020-RIFE/input_video
output_chunks = len([c for c in os.listdir('./output_chunks') if os.path.isfile(os.path.join("./output_chunks", c))])

with open("chunks_concat.txt", "w") as c:
  for i in range(output_chunks):
    c.write("file './output_chunks/chunk{}.mp4'\n".format(str(i)))

%cd output_chunks
!ffmpeg -f concat -safe 0 -i /content/arXiv2020-RIFE/input_video/chunks_concat.txt /content/arXiv2020-RIFE/output/final.mp4

In [ ]:
#@markdown ## Zip all output chunk (Optional)
!zip -r /content/arXiv2020-RIFE/output/chunks.zip /content/arXiv2020-RIFE/input_video/output_chunks

In [ ]:
#@markdown ## Copy Chunk to drive (Optional)
from google.colab import drive
drive.mount('/mnt/gdrive', force_remount=True)

!cp /content/arXiv2020-RIFE/output/chunks.zip /mnt/gdrive/MyDrive/